In [0]:
import warnings
warnings.filterwarnings("ignore")

In [0]:
import numpy as np
import pandas as pd

In [0]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings_2.csv')

In [89]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [90]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [0]:
ratings = ratings.iloc[:, 0:3]

In [92]:
ratings.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [93]:
movies.shape

(9742, 3)

In [94]:
ratings.shape

(100836, 3)

In [0]:
from sklearn.model_selection import train_test_split
training_set , test_set = train_test_split(ratings, test_size = 0.2)

In [96]:
training_set.shape

(80668, 3)

In [97]:
test_set.shape

(20168, 3)

In [0]:
## Converting into numpy arrays
training_set = np.array(training_set, dtype = 'int')
test_set = np.array(test_set, dtype = 'int')


In [0]:
# Getting the number of users and movies
max_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
max_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

In [100]:
print("Max users is : " + str(max_users))
print("Max movies is : " + str(max_movies))

Max users is : 610
Max movies is : 193609


In [0]:
# Converting the data into an array with users in lines and movies in columns

In [0]:
def convert(data):
  new_data = []
  for id_users in range(1, max_users + 1):
    id_movies = data[:,1][data[:,0] == id_users]
    id_ratings = data[:,2][data[:,0] == id_users]
    ratings = np.zeros(max_movies)
    ratings[id_movies - 1] = id_ratings
    new_data.append(list(ratings))
  return new_data

In [0]:
training_set = convert(training_set)

In [0]:
test_set = convert(test_set)

In [0]:
## Importing pytorch libraries:
import torch

In [0]:
# Converting the data into Torch tensors
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [0]:
# Converting the ratings into binary ratings 1 (Liked) or 0 (Not Liked)

# (Ratings (1, 2) -> 0, (3, 4, 5) -> 1, (Not rated -> -1))
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1

test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

In [0]:
# Importing other pytorch libraries
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [0]:
# Creating the architecture of the Restricted Boltzmann machines

# Creating an RBM class

class RBM():

  # Initializing the weights and biases randomly
  # nh - number of hidden nodes
  # nv - number of visible nodes
  # a - bias of hidden nodes w.r.t visible nodes
  # b - bias of visible nodes w.r.t hidden nodes
  # W - weights in the network 
  def __init__(self, nv, nh):
    self.W = torch.randn(nh, nv)
    
    self.a = torch.randn(1, nh)
    self.b = torch.randn(1, nv)

  # Gibs Sampling on the hidden nodes in the network
  def sample_h(self, x):
    wx = torch.mm(x, self.W.t())
    activation = wx + self.a.expand_as(wx)
    p_h_given_v = torch.sigmoid(activation)
    return p_h_given_v, torch.bernoulli(p_h_given_v)

  # Gibs sampling on the visible nodes in the network
  def sample_v(self, y):
    wy = torch.mm(y, self.W)
    activation = wy + self.b.expand_as(wy)
    p_v_given_h = torch.sigmoid(activation)
    return p_v_given_h, torch.bernoulli(p_v_given_h)

  # Training the dataset using Constrastive divergence
  # Using approximation of the Log-Likelihood Gradient
  def train(self, v0, vk, ph0, phk):
    self.W += (torch.mm(v0.t(),ph0) - torch.mm(vk.t(),phk)).t()
    self.b += torch.sum((v0 - vk), 0)
    self.a += torch.sum((ph0 - phk), 0)

  # Predict method for output 
  def predict( self, x): # x: visible nodes
    _, h = self.sample_h( x)
    _, v = self.sample_v( h)
    return v

In [0]:
nv = len(training_set[0])
nh = 100
batch_size = 100
rbm = RBM(nv, nh)

In [111]:
# Training the RBM
nb_epoch = 10
for epoch in range(1, nb_epoch + 1):
  train_loss = 0
  # Epoch Counter
  s = 0
  for id_user in range(0, max_users - batch_size, batch_size):

    vk = training_set[id_user:id_user+batch_size]
    v0 = training_set[id_user:id_user+batch_size]

    ph0,_ = rbm.sample_h(v0)

    for k in range(10):
      _,hk = rbm.sample_h(vk)
      _,vk = rbm.sample_v(hk)
      vk[v0<0] = v0[v0<0]

    phk,_ = rbm.sample_h(vk)

    rbm.train(v0, vk, ph0, phk)
    train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))

    s += 1
  print('No of Epoch : '+str(epoch)+' Total Loss : ' +str(train_loss/s)+ "\n")

No of Epoch : 1 Total Loss : tensor(0.3649)

No of Epoch : 2 Total Loss : tensor(0.2576)

No of Epoch : 3 Total Loss : tensor(0.2433)

No of Epoch : 4 Total Loss : tensor(0.2493)

No of Epoch : 5 Total Loss : tensor(0.2409)

No of Epoch : 6 Total Loss : tensor(0.2411)

No of Epoch : 7 Total Loss : tensor(0.2380)

No of Epoch : 8 Total Loss : tensor(0.2368)

No of Epoch : 9 Total Loss : tensor(0.2402)

No of Epoch : 10 Total Loss : tensor(0.2365)



In [112]:

# Testing the RBM
test_loss = 0
s = 0.

for id_user in range(max_users):

    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]

    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)

        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))

        s += 1.
print('Total test loss: '+ str(test_loss/s))

Total test loss: tensor(0.2526)


In [0]:
# Test set loss 0.25 -> System is 75% correctly recommending 

In [113]:
## Testing on One output 
# User no 33

y_pred = rbm.predict(test_set[33:34])
print(y_pred)

tensor([[1., 1., 1.,  ..., 0., 0., 1.]])


In [114]:
y_pred.shape

torch.Size([1, 193609])

In [0]:
## Converting torch tensor into list
y_pred_list = y_pred.numpy().tolist()

In [116]:
y_pred_list

[[1.0,
  1.0,
  1.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  1.0,
  1.0,
  0.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.0,
  1.0,
  1.0,
  0.0,
  0.0,
  1.0,
  0.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  0.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  1.0,
  1.0,
  0.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.0,
  0.0,
  1.0,
  1.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.0,
  1.0,
  0.0,
  0.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.0,
  1.0,
  1.0,
  0.0,
  0.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  1.0,
  1.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,

In [0]:
## Extracting movie indexes from list

In [0]:
movie_index = []
for i in range(1, max_movies):
  if y_pred_list[0][i] == 1:
    movie_index.append(i)

In [119]:
movie_index

[1,
 2,
 5,
 9,
 10,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 20,
 21,
 24,
 26,
 27,
 28,
 29,
 31,
 33,
 35,
 38,
 39,
 40,
 41,
 42,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 57,
 59,
 60,
 61,
 62,
 63,
 65,
 67,
 68,
 69,
 71,
 72,
 73,
 74,
 77,
 78,
 82,
 84,
 85,
 86,
 87,
 88,
 90,
 93,
 94,
 95,
 96,
 98,
 99,
 102,
 103,
 104,
 105,
 107,
 109,
 110,
 111,
 112,
 115,
 118,
 122,
 123,
 124,
 129,
 131,
 134,
 136,
 139,
 144,
 145,
 146,
 147,
 149,
 150,
 153,
 154,
 155,
 159,
 161,
 162,
 164,
 168,
 169,
 170,
 172,
 174,
 175,
 179,
 182,
 184,
 185,
 186,
 188,
 193,
 197,
 198,
 204,
 207,
 208,
 210,
 212,
 213,
 214,
 218,
 219,
 221,
 222,
 223,
 224,
 226,
 227,
 229,
 230,
 232,
 233,
 234,
 235,
 236,
 237,
 238,
 239,
 241,
 242,
 245,
 246,
 248,
 250,
 253,
 254,
 255,
 256,
 258,
 259,
 260,
 261,
 264,
 265,
 266,
 268,
 269,
 270,
 271,
 272,
 275,
 278,
 279,
 280,
 283,
 286,
 287,
 288,
 289,
 291,
 292,
 293,
 294,
 295,
 298,
 299,
 301

In [0]:
user_33_watched = ratings[ratings["userId"] == 33]["movieId"].tolist()

In [0]:
## Removing indexes of movies already watched

In [122]:
user_33_watched

[1,
 7,
 11,
 17,
 21,
 25,
 28,
 32,
 34,
 36,
 39,
 43,
 50,
 58,
 94,
 110,
 150,
 151,
 161,
 194,
 223,
 230,
 232,
 249,
 261,
 265,
 272,
 293,
 296,
 300,
 302,
 307,
 308,
 318,
 322,
 337,
 339,
 345,
 356,
 385,
 428,
 446,
 448,
 452,
 457,
 475,
 477,
 490,
 497,
 507,
 508,
 509,
 527,
 529,
 562,
 580,
 588,
 589,
 593,
 608,
 613,
 715,
 778,
 800,
 805,
 838,
 866,
 914,
 919,
 994,
 1027,
 1056,
 1059,
 1060,
 1095,
 1177,
 1183,
 1188,
 1210,
 1213,
 1245,
 1249,
 1271,
 1280,
 1357,
 1358,
 1365,
 1393,
 1399,
 1446,
 1476,
 1487,
 1500,
 1537,
 1542,
 1545,
 1580,
 1610,
 1614,
 1617,
 1621,
 1633,
 1639,
 1641,
 1643,
 1653,
 1658,
 1672,
 1673,
 1678,
 1682,
 1704,
 1719,
 1721,
 1777,
 1784,
 1799,
 1801,
 1805,
 1841,
 1885,
 1888,
 1918,
 1923,
 2028,
 2058,
 2080,
 2087,
 2108,
 2112,
 2268,
 2272,
 2291,
 2294,
 2302,
 2310,
 2324,
 2333,
 2336,
 2355,
 2357,
 2391,
 2396,
 2443,
 2485,
 2490,
 2501,
 2539,
 2542,
 2571,
 2580,
 2690,
 2724,
 2762,
 2875,
 2

In [0]:
for item in user_33_watched:
  if item in movie_index:
    movie_index.pop(item)

In [124]:
movies.dtypes

movieId     int64
title      object
genres     object
dtype: object

In [125]:
## Recommendations for user 33

movies[movies['movieId'].isin(movie_index)]["title"]

0                            Toy Story (1995)
4          Father of the Bride Part II (1995)
8                         Sudden Death (1995)
9                            GoldenEye (1995)
11         Dracula: Dead and Loving It (1995)
                        ...                  
9690                 Making a Murderer (2015)
9693          Fullmetal Alchemist 2018 (2017)
9707    Jurassic World: Fallen Kingdom (2018)
9718                        Boundaries (2018)
9728                      Liquid Truth (2017)
Name: title, Length: 3293, dtype: object